<h1 align = "center"><b> Facebook Prophet </b></h1>

&emsp;&emsp; Este estudo de caso aborda os fundamentos da análise de séries temporais. O objetivo é construir dois modelos capazes de prever a demanda de um produto e, assim, auxiliar uma empresa a otimizar suas decisões logísticas.

&emsp;&emsp; Nesta etapa, vamos analisar e transformar os dados para que possamos aplicar o modelo Facebook Prophet.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6


In [3]:
tabela_vendas = pd.read_csv('../Demanda Diária.csv', index_col = 0)
tabela_vendas = tabela_vendas.loc[tabela_vendas['loja'] == 1]
tabela_vendas = tabela_vendas.loc[tabela_vendas['produto'] == 1]
tabela_vendas = tabela_vendas.drop(['loja', 'produto'], axis = 1)

série_vendas = pd.Series(tabela_vendas['itens_vendidos'], index = tabela_vendas.index)

<h2 align = "center"><b> Criando o Modelo </b></h2>

&emsp;&emsp; O prophet impõe que os dados estejam em um dataframe com duas colunas (ds e y). A coluna ds deve conter as datas em formato YYYY-MM-DD e a coluna y deve conter os valores que queremos prever.

In [9]:
from prophet import Prophet

In [39]:
tabela_prophet = pd.DataFrame({'ds': série_vendas.index, 'y': série_vendas.values})

tabela_prophet_treino = tabela_prophet.iloc[:-31] # Retirando o último mês do treino
tabela_prophet_teste = tabela_prophet.iloc[-31:] # Utilizando o último mês como teste

In [40]:
modelo = Prophet()
modelo.fit(tabela_prophet_treino)

16:39:30 - cmdstanpy - INFO - Chain [1] start processing
16:39:31 - cmdstanpy - INFO - Chain [1] done processing


In [44]:
future = modelo.make_future_dataframe(periods = 31, freq = 'D')
previsão = modelo.predict(future)

In [54]:
resultado = pd.concat([previsão['yhat'].tail(31), tabela_prophet_teste['y']], axis = 1).reset_index(drop = True)

resultado.rename(columns={'yhat': 'Previsão', 'y': 'Valor Real'}, inplace=True)


In [63]:
display(resultado)

,Previsão,Valor Real
0,21.452456,19
1,22.882720,16
2,23.232690,31
3,14.585891,7
4,16.843408,20
5,17.052788,17
6,17.358283,18
7,18.652758,17
8,20.225551,27
9,20.748260,20


In [56]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [59]:
print('RMSE: ', mean_squared_error(resultado['Valor Real'], resultado['Previsão'], squared = False))
print('MAE: ', mean_absolute_error(resultado['Valor Real'], resultado['Previsão']))

RMSE:  4.602214728606465
MAE:  3.547750814940637


In [ ]:
modelo.plot_components(previsão)

![plot_components](./Gráficos/plot_components.png)